In [ ]:
from IPython import get_ipython
from IPython.display import display

import os
from google.colab import userdata

In [ ]:
!pip install google-genai

In [ ]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai

client = genai.Client()

In [ ]:
for model in client.models.list():
  print(model.name)

In [ ]:
model = "gemini-2.5-flash-preview-04-17"

resposta = client.models.generate_content(model=model, contents= "Como poderia otimizar seu tempo nas compras?")

In [ ]:
resposta.text

In [ ]:
from google.genai import types

chat_config = types.GenerateContentConfig(
    system_instruction= "Você é um assistente pessoal que sempre respode de forma sucinta e leve.",
)

In [ ]:
chat = client.chats.create(model=model, config=chat_config)

resposta = chat.send_message("Qual a funcionalidade de criar um gerenciador de lista de compras?")


In [ ]:
resposta.text

In [ ]:
#--- Bem-vindo ao Gerenciador de Lista de Compras 🛒! ---
#### Este programa vai te ajudar a lembrar o que comprar 🤔😄.
# #### Aqui guardamos nossa lista📜.
# #### É como um caderninho mágico: o nome do produto é a chave, e a quantidade é o valor.

lista_de_compras = {}
def mostrar_mensagem_de_abertura():
    print("Oi! 👋 Sou seu assistente de Lista de Compras !")
    print("Vamos anotar tudo para não esquecer nada no mercado!")
    print("\nPara começar, digite 'sim' seguido do item e quantidade (ex: quero 3 bananas)!")
    print("Se precisar de ajuda com outros comandos (como tirar, ver a lista, etc.), digite 'ajuda'.")

    mostrar_ajuda() # Garante que a ajuda é mostrada logo no início

def mostrar_ajuda():
    print("Aqui estão algumas dicas para te ajudar:")
    print("                                        ")
    print("➡️ Para colocar algo na lista:")
    print("   1 banana | quero 2 banana | adicione  3 bananas  ")
    print("                                         ")
    print("➡️ Para tirar algo da lista:  ")
    print("    tirar banana | remove banana | tire 1 banana")
    print("                                                 ")
    print("➡️ Para ver sua lista:                  ")
    print("    lista                                ")
    print("                                         ")
    print("➡️ Para remover todos os produtos:       ")
    print("    remover tudo                          ")
    print("                                         ")
    print("➡️ Para ver esta ajuda de novo:         ")
    print("    ajuda                                ")
    print("                                          ")
    print("➡️ Para fechar o programa:               ")
    print("    sair                                  ")
    print("==================================\n      ")

# Esta função não é usada no loop principal atual, mas pode ser útil.
# Deixei ela aqui, mas o loop principal usa 'mostrar_lista'.
def marcar_como_comprado(entrada_do_usuario):
    """Marca um item como comprado"""
    if not entrada_do_usuario:
        print("❓ Qual item você pegou? Ex: 'peguei bananas' ou 'peguei 2 maçãs'")
        return

    # Simplificado para pegar apenas o nome do item após o comando de "pegar"
    # Pode ser necessário ajustar esta lógica se você quiser suportar "peguei 2 maçãs"
    # e diminuir a quantidade ao invés de apenas marcar como comprado.
    # Por enquanto, apenas marca como comprado baseado no nome do item.
    item = entrada_do_usuario.lower()

    if item not in lista_de_compras:
        print(f"🤷‍♀️ '{item}' não está na lista!")
        return

    # Altera o status 'comprado' para True
    # Verifica se o item é um dicionário antes de acessar a chave 'comprado'
    if isinstance(lista_de_compras.get(item), dict):
         lista_de_compras[item]["comprado"] = True
         print(f"✅ Marcado como comprado: {item.title()}")
    else:
         print(f"⚠️ Não foi possível marcar '{item.title()}' como comprado (formato inválido na lista).")


# Renomeada para mostrar o status de comprado
# Esta função não é usada no loop principal atual
def mostrar_lista_com_status():
    """Mostra todos os itens da lista de compras com status"""
    if not lista_de_compras:
        print("\n🛒 Lista vazia! Adicione itens primeiro.")
    else:
        print("\n=== 🛍️ LISTA DE COMPRAS (✅ = comprado) ===")
        for item, dados in lista_de_compras.items():
            # Acessa o status 'comprado', padrão para False se a chave não existir ou não for dict
            status = "✅" if isinstance(dados, dict) and dados.get("comprado", False) else "⬜"
            # Acessa a quantidade, padrão para '?' se não for dict com 'quantidade'
            quantidade_str = str(dados.get("quantidade", "?")) if isinstance(dados, dict) else str(dados) if isinstance(dados, int) else "?"
            print(f"{status} {item.title()}: {quantidade_str}")
        print("========================================")

# Funções adicionar_item, remover_item e mostrar_lista corrigidas
def adicionar_item(entrada_do_usuario):
    if not entrada_do_usuario:
        print("❓ O que você gostaria de adicionar? Me diga! (Exemplo: '3 chocolates' ou 'quero 3 chocolates')")
        return

    # Remove os prefixos de adição se presentes
    prefixes_adicionar = ["sim ", "sim", "adicionar ", "adicione ", "quero "]
    for prefixo in prefixes_adicionar:
        if entrada_do_usuario.lower().startswith(prefixo):
            entrada_do_usuario = entrada_do_usuario[len(prefixo):].strip()
            break

    if not entrada_do_usuario:
         print("❓ Você esqueceu de dizer o item e/ou quantidade depois do comando de adição.")
         return

    partes = entrada_do_usuario.split(maxsplit=1) # Divide no máximo uma vez para separar quantidade e item
    nome_item_partes = partes
    quantidade = 1  # Valor padrão

    # Tenta extrair a quantidade se o primeiro termo for um número
    if partes and partes[0].isdigit():
        try:
            quantidade = int(partes[0])
            if quantidade <= 0:
                print("Oops! 😬 A quantidade tem que ser 1 ou mais. Tente de novo!")
                return
            if len(partes) > 1:
                nome_item_partes = partes[1].split() # O nome do item é o resto
            else:
                 print("❓ Você disse uma quantidade, mas não o item. Ex: 'adicionar 2 bananas'")
                 return
        except ValueError:
            # Se o primeiro elemento não for um número válido, trata tudo como nome
            quantidade = 1
            nome_item_partes = partes
    else:
        # Se não começar com número, a quantidade é 1
        quantidade = 1
        nome_item_partes = partes


    if not nome_item_partes:
        print("❓ Você esqueceu de dizer o nome do item. Exemplo: '3 bananas' ou 'bananas'")
        return

    item_chave = " ".join(nome_item_partes).lower()

    # Garante que a entrada na lista_de_compras é um dicionário com 'quantidade'
    # Se o item já existe e não está no formato de dicionário, converte
    if item_chave in lista_de_compras and (not isinstance(lista_de_compras[item_chave], dict) or 'quantidade' not in lista_de_compras[item_chave]):
         lista_de_compras[item_chave] = {"quantidade": lista_de_compras.get(item_chave, 0), "comprado": False}


    if item_chave in lista_de_compras:
        lista_de_compras[item_chave]["quantidade"] += quantidade
        print(f"👍 Adicionei mais {quantidade} '{item_chave.title()}'. Total: {lista_de_compras[item_chave]['quantidade']}")
    else:
        lista_de_compras[item_chave] = {"quantidade": quantidade, "comprado": False}
        print(f"✅ Adicionado: {quantidade} '{item_chave.title()}' foi para a lista!")


def remover_item(entrada_do_usuario):
    """Remove um item da lista de compras ou diminui sua quantidade."""
    if not entrada_do_usuario:
        print("❓ O que você quer tirar da lista? Ex: 'tirar 1 chocolate' ou 'tirar chocolate'")
        return

    # Remove prefixes like "tirar", "remover", etc.
    prefixes_remover = ["tirar ", "tire ", "remove ", "remover "] # Adiciona espaço para garantir que é o prefixo
    entrada_sem_prefixo = entrada_do_usuario.lower()
    prefixo_encontrado = None
    for prefixo in prefixes_remover:
        if entrada_sem_prefixo.startswith(prefixo):
            entrada_sem_prefixo = entrada_sem_prefixo[len(prefixo):].strip()
            prefixo_encontrado = prefixo # Marca que um prefixo foi encontrado
            break

    # Lida com casos onde o prefixo não tem espaço (ex: "tirarbanana")
    if not prefixo_encontrado:
         prefixes_remover_sem_espaco = ["tirar", "tire", "remove", "remover"]
         for prefixo in prefixes_remover_sem_espaco:
             if entrada_sem_prefixo.startswith(prefixo):
                 entrada_sem_prefixo = entrada_sem_prefixo[len(prefixo):].strip()
                 break


    if not entrada_sem_prefixo:
         print("❓ Qual item você quer remover depois de usar o comando?")
         return

    partes = entrada_sem_prefixo.split(maxsplit=1)
    quantidade_para_remover = 0 # Usamos 0 para indicar que queremos remover tudo por padrão
    nome_item_partes = partes

    # Try to parse quantity at the beginning
    if partes and partes[0].isdigit():
        try:
            quantidade_para_remover = int(partes[0])
            if quantidade_para_remover <= 0:
                print("Oops! 😬 A quantidade para tirar tem que ser 1 ou mais.")
                return
            if len(partes) > 1:
                nome_item_partes = partes[1].split() # Item name is the rest
            else:
                 # User only typed a number after the remove command (e.g., "remove 3")
                 print("❓ Você disse uma quantidade, mas não o item. Ex: 'tirar 2 bananas'")
                 return
        except ValueError:
             # Should not happen with isdigit() check, but for safety
             print("❌ Quantidade inválida.")
             return
    else:
        # No quantity specified, remove all of the item
        quantidade_para_remover = 0 # Explicitly set to 0 for "remove all" logic
        nome_item_partes = partes # O nome do item é a entrada sem prefixo

    if not nome_item_partes:
        print("❓ Hum... qual item você quer tirar? Tente: tirar suco de uva")
        return

    item_chave = " ".join(nome_item_partes).lower()

    # Find the actual key in the dictionary (case-insensitive match)
    item_encontrado = None
    for key in lista_de_compras.keys():
        if key.lower() == item_chave:
            item_encontrado = key
            break

    if not item_encontrado:
        print(f"🤔 Ué? '{item_chave.title()}' nem estava na lista...")
        return

    # Ensure the item structure is the new dictionary format
    if not isinstance(lista_de_compras[item_encontrado], dict) or "quantidade" not in lista_de_compras[item_encontrado]:
        # If it's in the old int format or malformed, remove it completely
        print(f"⚠️ Formato antigo ou inválido encontrado para '{item_encontrado.title()}'. Removendo completamente.")
        del lista_de_compras[item_encontrado]
        return

    # Now handle removal based on parsed quantity
    if quantidade_para_remover == 0 or quantidade_para_remover >= lista_de_compras[item_encontrado]["quantidade"]:
        # Remove all of the item
        quantidade_total_anterior = lista_de_compras[item_encontrado]["quantidade"]
        del lista_de_compras[item_encontrado]
        print(f"🗑️ Pronto! Tirei todos os {quantidade_total_anterior} '{item_encontrado.title()}' da lista.")
    else:
        # Remove a specific quantity
        lista_de_compras[item_encontrado]["quantidade"] -= quantidade_para_remover
        print(f"👍 Tirei {quantidade_para_remover} '{item_encontrado.title()}'. Ainda tem {lista_de_compras[item_encontrado]['quantidade']} na lista.")


# Função mostrar_lista corrigida
def mostrar_lista():
    """Mostra todos os itens da lista de compras."""
    if not lista_de_compras:
        print("\n Sua lista de compras está vazia! 🛒💨 Parece que não precisamos comprar nada ainda!")
    else:
        print("\n=== 🛍️ SUA LISTA DE COMPRAS ATUAL 🛍️ ===")
        for item,dados in lista_de_compras.items():
          # Garante que a entrada é um dicionário para acessar 'quantidade'
          if isinstance(dados, dict) and 'quantidade' in dados:
            print(f"➡️ {item.title()}: {dados['quantidade']} unidade(s)")
          else:
             # Se for um formato antigo (apenas int), mostra assim (embora o adicionar_item
             # e remover_item agora convertam para dict)
             print(f"➡️ {item.title()}: {dados} unidade(s) (formato antigo/inconsistente)")
        print("==========================================")
    print()

# --- Inicio ---

# Loop para esperar o usuário digitar 'sim' para iniciar
mostrar_mensagem_de_abertura()
while True:
    try:
        iniciar_comando = input("Digite 'sim' para iniciar ou 'sair' para fechar: \n> ").strip().lower()
        if iniciar_comando == 'sim':
            break  # Sai do loop de inicialização
        elif iniciar_comando == 'sair':
            print("Fechando a lista... Tchau! 👋")
            exit()  # Sai completamente do programa
        else:
            print("Comando inválido. Por favor, digite 'sim' ou 'sair'.")
    except KeyboardInterrupt:
        print("\nFechando a lista... Tchau! 👋")
        exit()

# --- Inicio do Loop Principal de Comandos ---
lista_de_compras = {} # Reinicia a lista de compras para a sessão principal

while True:
    try:
        comando_usuario = input("Qual o próximo comando? (Digite 'ajuda' se precisar) \n> ").strip()

        if not comando_usuario:
            continue

        comando_lower = comando_usuario.lower()

        # 1. Comando SAIR
        if comando_lower == "sair":
            print("Até a próxima! Boas compras! 😄")
            break

        # 2. Comando AJUDA
        elif comando_lower == "ajuda":
            mostrar_ajuda()

        # 3. Comando LISTA
        # Agora usa a função mostrar_lista corrigida
        elif comando_lower in ["lista", "ver lista", "l"]:
           mostrar_lista()

        # 4. Comando LIMPAR LISTA
        elif comando_lower in ["limpar lista", "remover tudo"]:
            lista_de_compras.clear()
            print("🧹✨ Prontinho! Sua lista de compras foi esvaziada.")

        # 5. Comandos de ADIÇÃO (com prefixo ou numérico)
        # Combina as lógicas de adição que começam com prefixo ou número
        elif comando_lower.startswith(("adicionar ", "adicione ", "quero ")) or comando_usuario[:1].isdigit() or comando_lower.startswith("sim"):
             adicionar_item(comando_usuario)

        # 6. Comandos de REMOÇÃO
        # Chama remover_item e ela lida com os prefixes internamente
        elif any(comando_lower.startswith(p) for p in ["tirar", "tire", "remove", "remover"]):
            remover_item(comando_usuario)

        # 7. Comando de Marcar como comprado (opcional, não estava no loop principal original)
        # Adicionado baseado na função que você tinha criado
        elif comando_lower.startswith(("peguei ", "comprado ")):
             item_para_marcar = comando_usuario.split(maxsplit=1)[1].strip() if len(comando_usuario.split(maxsplit=1)) > 1 else ""
             marcar_como_comprado(item_para_marcar)


        # 8. Comando não reconhecido
        else:
            print("😕 Comando não reconhecido. Digite 'ajuda' para ver as opções.")

    except KeyboardInterrupt:
        print("\nFechando a lista... Tchau! 👋")
        break

Oi! 👋 Sou seu assistente de Lista de Compras !
Vamos anotar tudo para não esquecer nada no mercado!

Para começar, digite 'sim' seguido do item e quantidade (ex: quero 3 bananas)!
Se precisar de ajuda com outros comandos (como tirar, ver a lista, etc.), digite 'ajuda'.
Aqui estão algumas dicas para te ajudar:
                                        
➡️ Para colocar algo na lista:
   1 banana | quero 2 banana | adicione  3 bananas  
                                         
➡️ Para tirar algo da lista:  
    tirar banana | remove banana | tire 1 banana
                                                 
➡️ Para ver sua lista:                  
    lista                                
                                         
➡️ Para remover todos os produtos:       
    remover tudo                          
                                         
➡️ Para ver esta ajuda de novo:         
    ajuda                                
                                          
➡️ Para fechar